In [1]:
# import general packages:
import numpy as np
import pandas as pd
from pandas import Timestamp
import os
import datetime as dt
import time 
import itertools
from math import sqrt
import csv
from datetime import datetime
import timestring
from IPython.core.display import display, HTML
from IPython.display import Image
display(HTML("<style>.container { width:100% !important; }</style>"))

# import visualization packages:
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly import tools

In [2]:
import tkinter as tk
from tkinter import filedialog

In [3]:
# standardize dataframe colume names
def col_name(df):
    """
    this is to trim the data_frame column names to a unique format:
    all case, replace space to underscore, remove parentheses
    param df:
        raw from share drive for
    return:
        polished data set with new column names
    """
    df.columns = df.columns.str.strip().str.lower().str.replace('-','').str.replace(' ', '_').str.replace('(', '').\
                    str.replace(')', '').str.replace('"','')
    return df


def preprocess_fill_na(df):
    df_all = df.copy()
    df_price = df_all[list(df_all.loc[:, '1_piece_bucket': 'standard_cost'])]
    df_price = df_all.loc[:, '1_piece_bucket': 'standard_cost'].replace({0:np.nan})
    for index, row in df_price.iterrows():
        if row.isnull().all():
            df_all.loc[index, '1_piece_bucket': '1001_piece_bucket'] = df_all.loc[index, 'standard_cost']
        elif not np.isnan(row[0]) :
            df_all.loc[index, '1_piece_bucket': '1001_piece_bucket'] = row.ffill()
        else:
            valid_col_index = row.first_valid_index()
            df_all.loc[index, '1_piece_bucket': valid_col_index] = row[valid_col_index]
            df_all.loc[index, valid_col_index:] = row[valid_col_index:].ffill()
    return df_all


def get_vop_qty(pricing_row, vop_day = 1):
    """
    pricing_row: data series, single entire row from df_result
    vop_days: iterating from days selection
    """
    vop_qty = (pricing_row['eau']/320)*vop_day
    return int(vop_qty) + 1

def get_order_qty(pricing_row, vop_qty):
    """
    pricing_row: data series, single entire row from df_result
    vop_qty: iterated qty from VOP days calculation
    """
    mlpq = pricing_row['multiple_order_qty'] 
    minq = pricing_row['minimum_reorder_qty']

    if (mlpq == 0) & (minq == 0):
        vopq = vop_qty
    
    elif (mlpq == 0) & (minq != 0):
        vopq = max(vop_qty, minq)
    
    elif (minq == 0) & (mlpq != 0):
        if vop_qty <= mlpq:
            vopq = mlpq
        else:
            vopq = ((vop_qty // mlpq)+1) * mlpq
            
    else:
        if mlpq <= minq: 
            if vop_qty <= minq:
                vopq = ((minq // mlpq)+1) * mlpq
            else:
                vopq = ((vop_qty // mlpq)+1) * mlpq    
                
        else:
            if vop_qty < mlpq:
                vopq = mlpq
            else:
                vopq = ((vop_qty // mlpq)+1) * mlpq                
    return vopq 


def get_unit_cost(pricing_row, pricing_break, order_qty):
    """
    pricing_row: data series, single entire row from df_result
    pricing_break: dictionary: pricing category with breaking range
    order_qrt: is vopq after comapring with MOQ & MINQ iterated qty from VOP days calculation
    
    return: unit_cost: pricing from break bucket
    """
    for key, value in pricing_break.items():
        if (value[0] <= order_qty) & (order_qty < value[1]):
            unit_cost = pricing_row[key]            
            return key, unit_cost           
    return None


def get_purchasing_cost(unit_cost, pricing_row):
    """
    unit_cost: int, result from pricing break
    pricing_row: data series, single entire row from df_result
    return: total purchasing cost per item
    """
    purchasing_cost = unit_cost * pricing_row['eau']
    return round(purchasing_cost,2)


def get_order_frequency(order_qty, pricing_row):
    """
    order_qty: ordering qty result from function get_order_qty
    pricing_row: data series, single entire row from df_result
    return: 
    """
    order_frequency = min(pricing_row['eau']/order_qty, 52)
    return round(order_frequency,2)


def get_holding_cost(purchasing_cost, order_frequency, financial_rate=0.125):
    financial_carrying_frequency = max(order_frequency, 365/83)
    holding_cost = purchasing_cost * financial_rate / financial_carrying_frequency 
    return round(holding_cost, 2)


def get_logistic_cost(order_frequency):
    logistic_cost = order_frequency*38/5
    return round(logistic_cost,2)

In [4]:
def get_optimal_vops(pricing_row, cost_plot = False, summary = True, financial_rate = 0.1):
    
    cost_dict = {'purchasing_cost': [],
                'holding_cost' : [],
                'logistic_cost' : [],
                'total_cost' : []}  
    
    unit_cost_list = []
    
    min_combined_cost = np.inf
    vop_day = 1
    optimal_unit_cost = 0
    min_purchasing_cost = 0
    min_holding_cost = 0
    min_logistic_cost = 0
    min_combined_cost = np.inf
    optimal_vop_day = 0
    optimal_vop_qty = 0
    optimal_vop_freq = 0
    optimal_pricing_bucket = 0
    
#     while vop_day < 84:
    for vop_day in range (7, 84): # due to maximum order freqency is 52, hence vop_day starts on 7 days

        vop_qty = get_vop_qty(pricing_row=pricing_row, vop_day = vop_day)
        order_qty = get_order_qty(pricing_row=pricing_row, vop_qty = vop_qty)
        pricing_bucket, unit_cost = get_unit_cost(pricing_row, pricing_break, order_qty=order_qty)
        purchasing_cost = get_purchasing_cost(unit_cost = unit_cost, pricing_row= pricing_row)
        order_frequency = get_order_frequency(order_qty, pricing_row)
        holding_cost = get_holding_cost(purchasing_cost, order_frequency=order_frequency, financial_rate=financial_rate)
        logistic_cost = get_logistic_cost(order_frequency)
        combined_cost = round((purchasing_cost + holding_cost + logistic_cost), 2)
        if combined_cost <= min_combined_cost:
            min_purchasing_cost = purchasing_cost
            min_holding_cost = holding_cost
            min_logistic_cost = logistic_cost
            min_combined_cost = combined_cost
            optimal_vop_day = vop_day
            optimal_vop_qty = order_qty
            optimal_unit_cost = unit_cost
            optimal_vop_freq = order_frequency
            optimal_pricing_bucket = pricing_bucket
        
        unit_cost_list.append(unit_cost)

        cost_dict['purchasing_cost'].append(purchasing_cost)
        cost_dict['holding_cost'].append(holding_cost)
        cost_dict['logistic_cost'].append(logistic_cost)
        cost_dict['total_cost'].append(combined_cost)
        
#         vop_day += 1
    
    cost_df = pd.DataFrame(cost_dict)
        
    if cost_plot:
        x_vops = list(np.arange(7,84))
        fig = make_subplots(rows=4, cols=1)
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['holding_cost'], mode = 'lines', name = 'holding_cost'))
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['logistic_cost'], mode = 'lines', name = 'logistic_cost'))
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['purchasing_cost'], mode = 'lines', name = 'purchasing_cost'), row=2, col=1)
        fig.add_trace(go.Scatter(x = x_vops, y = cost_df['total_cost'], mode = 'lines', name = 'total_cost'), row=3, col=1)
        fig.add_trace(go.Scatter(x = x_vops, y = unit_cost_list, mode = 'lines', name = 'unit_cost'), row=4, col=1)
        fig.show()
        
    if summary:
        print(f'Material: {pricing_row.part_number}, {pricing_row.part_description}')
        print(f'vop_day: {optimal_vop_day} (days cycle)')
        print(f'EAU: {pricing_row.eau} (ea/year)\nEach order qty:{optimal_vop_qty} (ea/order)\norder frequency: {optimal_vop_freq} (times/year)')
        print(f'pricing bucket: {optimal_pricing_bucket}\nunit cost: {optimal_unit_cost}')
        print(f'material purchasing cost: {min_purchasing_cost} (dollars/year)\nholding cost:{min_holding_cost} (dollars/year)\nlogistic cost:{min_logistic_cost} (dollars/year)')
        print(f'total_landing_cost_optimal: {min_combined_cost} (dollars/year)\n')

    return optimal_vop_day, optimal_vop_qty, optimal_vop_freq, optimal_unit_cost

## Start to run model 

In [11]:
# file selection
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [12]:
file_path

'S:/OSK-Share/DEPT/LOGISTICS/LC3/Projects/Logistics Optimization Tool/Files for Nate to Upload - VOP/VOP_Uploading/defense_uploading/pricing_loading_master_0 VOP Project 2.xlsx'

In [25]:
pd.set_option('display.max_columns', None)
df_pricing = col_name(pd.read_excel(file_path))
df_result = preprocess_fill_na(df_pricing)

In [27]:
df_result.head(20)

,part_number,part_description,eau,minimum_reorder_qty,multiple_order_qty,1_piece_bucket,2_piece_bucket,6_piece_bucket,16_piece_bucket,31_piece_bucket,51_piece_bucket,101_piece_bucket,251_piece_bucket,501_piece_bucket,1001_piece_bucket,standard_cost
0,12422851,"DEVICE,CONTROL,LOAD AND BATT",1380,0,12,560.3706,560.3706,560.3706,560.3706,560.3706,560.3706,560.3706,560.3706,560.3706,560.3706,560.3706
1,12423713,"ALTERNATOR 260 AMP, DUAL VOLTA",0,0,0,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100
2,12423713,"ALTERNATOR 260 AMP, DUAL VOLTA",1368,0,12,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100,2290.9100
3,12601972-T01,"ALT,28,650,KEY,EAR,J1939",0,0,0,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400
4,12601972-T01,"ALT,28,650,KEY,EAR,J1939",4083,0,3,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400,3550.7400
5,12611926,"ALT,28,740,EAR,J1939",5,0,0,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692
6,12611926,"ALT,28,740,EAR,J1939",0,0,0,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692,4931.6692
7,22KP939,"EYEBOLT,CLOSED .38-16X2.50 ZC",200,0,0,1.7672,1.7672,1.7672,1.7672,1.7672,1.7672,1.7672,1.7672,1.7672,1.7672,1.7672
8,3590302,"ALTERNATOR,260A 24VDC W-PULLEY",0,0,0,2563.8900,2563.8900,2563.8900,2563.8900,2285.2000,2285.2000,2285.2000,2225.3400,2225.3400,2225.3400,2225.3400
9,3590302,"ALTERNATOR,260A 24VDC W-PULLEY",708,0,12,2563.8900,2563.8900,2563.8900,2563.8900,2285.2000,2285.2000,2285.2000,2225.3400,2225.3400,2225.3400,2225.3400


In [15]:
sum(df_result.loc[:,'1_piece_bucket': 'standard_cost'].isna().sum())

0

In [16]:
df_result[df_result.loc[:,'1_piece_bucket': 'standard_cost'].isna().any(axis = 1)]

,part_number,part_description,eau,minimum_reorder_qty,multiple_order_qty,1_piece_bucket,2_piece_bucket,6_piece_bucket,16_piece_bucket,31_piece_bucket,51_piece_bucket,101_piece_bucket,251_piece_bucket,501_piece_bucket,1001_piece_bucket,standard_cost


In [5]:
# quantity break for pricing break 
pricing_break = {'1_piece_bucket': [1,2],
 '2_piece_bucket': [2,6],
 '6_piece_bucket': [6, 16],
 '16_piece_bucket': [16, 31],
 '31_piece_bucket': [31, 51],
 '51_piece_bucket': [51, 101],
 '101_piece_bucket': [101, 251],
 '251_piece_bucket': [251, 501],
 '501_piece_bucket': [501, 1001],
 '1001_piece_bucket': [1001, np.inf]}

## Single item process >>>

In [28]:
random_pick = np.random.randint(-1, df_result.shape[0]-1)
single_item = df_result.copy().iloc[16,:]
single_item

part_number                             12601058
part_description       MITER BOX,STEERING,PHASED
eau                                         4573
minimum_reorder_qty                            0
multiple_order_qty                             4
1_piece_bucket                            180.82
2_piece_bucket                            180.82
6_piece_bucket                            180.82
16_piece_bucket                           180.82
31_piece_bucket                           180.82
51_piece_bucket                           180.82
101_piece_bucket                          180.82
251_piece_bucket                          180.82
501_piece_bucket                          180.82
1001_piece_bucket                         180.82
standard_cost                             180.82
Name: 16, dtype: object

### Optimal buy policy

In [29]:
defense_financial_rate = 0.073
get_optimal_vops(pricing_row=single_item,cost_plot=True, summary=True, financial_rate=defense_financial_rate)

Material: 12601058, MITER BOX,STEERING,PHASED
vop_day: 7 (days cycle)
EAU: 4573 (ea/year)
Each order qty:104 (ea/order)
order frequency: 43.97 (times/year)
pricing bucket: 101_piece_bucket
unit cost: 180.82
material purchasing cost: 826889.86 (dollars/year)
holding cost:1372.82 (dollars/year)
logistic cost:334.17 (dollars/year)
total_landing_cost_optimal: 828596.85 (dollars/year)



(7, 104, 43.97, 180.82)

## Process in batch material file

In [16]:
vop_master_run = df_result.copy()

In [17]:
vop_master_run

,part_number,part_description,eau,minimum_reorder_qty,multiple_order_qty,1_piece_bucket,2_piece_bucket,6_piece_bucket,16_piece_bucket,31_piece_bucket,51_piece_bucket,101_piece_bucket,251_piece_bucket,501_piece_bucket,1001_piece_bucket,standard_cost
0,3639605,"HOOD LOCK,WLDMT",320,0,5,104.45,68.51,44.55,37.02,34.89,33.98,33.29,33.29,33.29,33.29,34.8900
1,12412999,"PIVOT ARM, RH",3,0,8,156.89,132.19,107.49,95.14,92.67,87.73,82.79,82.79,82.79,82.79,134.6196
2,3973985,"ADAPTER,REAR,WLDMT",20,0,0,171.75,126.13,95.71,86.20,83.44,82.29,81.40,80.86,80.68,80.59,86.8120
3,12505369,"HEAD, PIVOT BEARING CARRIER",576,0,48,58.73,49.10,43.35,41.55,40.64,38.66,36.40,36.40,36.40,36.40,36.6584
4,3975760,"ANGLE,REAR CAB MOUNT",672,0,48,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.4099
5,1330100,"CHAN,DECK SPRT",7,0,0,55.33,55.33,37.28,31.63,30.00,29.31,27.78,27.78,27.78,27.78,37.2800
6,3696217,PUMP SUPPORT WELDMENT,144,0,18,72.90,72.90,54.16,52.06,51.99,51.96,51.94,51.94,51.94,51.94,52.0600
7,3706642,"CAB MT,RR,RH,WLDMT",312,0,6,351.31,287.10,244.29,230.92,226.52,225.40,221.16,220.61,219.41,218.00,226.5200
8,3724574,"CAB MT,REAR,RH WLDMT",186,0,0,347.39,283.18,240.37,227.00,222.59,221.49,220.24,218.94,217.54,216.14,240.3700
9,3724584,"CAB MT,REAR,LH WLDMT",186,0,0,344.94,281.53,239.26,226.05,221.74,220.61,219.38,217.98,216.58,215.28,239.2600


In [18]:
vop_master_run[['optimal_vop_day', 'optimal_vop_qty', 'optimal_vop_freq']] = vop_master_run.apply(lambda row: get_optimal_vops(row, financial_rate=defense_financial_rate) if row['eau'] != 0 else 0, axis=1, result_type='expand')

Material: 3639605, HOOD LOCK,WLDMT
vop_day: 53 (days cycle)
EAU: 320 (ea/year)
Each order qty:55 (ea/order)
order frequency: 5.82 (times/year)
pricing bucket: 51_piece_bucket
unit cost: 33.98
material purchasing cost: 10873.6 (dollars/year)
holding cost:136.39 (dollars/year)
logistic cost:44.23 (dollars/year)
total_landing_cost_optimal: 11054.22 (dollars/year)

Material: 12412999, PIVOT ARM, RH
vop_day: 83 (days cycle)
EAU: 3 (ea/year)
Each order qty:8 (ea/order)
order frequency: 0.38 (times/year)
pricing bucket: 6_piece_bucket
unit cost: 107.49
material purchasing cost: 322.47 (dollars/year)
holding cost:61.95 (dollars/year)
logistic cost:2.89 (dollars/year)
total_landing_cost_optimal: 387.31 (dollars/year)

Material: 3973985, ADAPTER,REAR,WLDMT
vop_day: 83 (days cycle)
EAU: 20 (ea/year)
Each order qty:6 (ea/order)
order frequency: 3.33 (times/year)
pricing bucket: 6_piece_bucket
unit cost: 95.71
material purchasing cost: 1914.2 (dollars/year)
holding cost:41.96 (dollars/year)
logisti

Material: 12645689-001-T02, BRKT,STOP BUMPER,RH,WLDMT
vop_day: 83 (days cycle)
EAU: 15 (ea/year)
Each order qty:4 (ea/order)
order frequency: 3.75 (times/year)
pricing bucket: 2_piece_bucket
unit cost: 47.68
material purchasing cost: 715.2 (dollars/year)
holding cost:13.92 (dollars/year)
logistic cost:28.5 (dollars/year)
total_landing_cost_optimal: 757.62 (dollars/year)

Material: 12645689-002-T02, BRKT,STOP BUMPER,LH,WLDMT
vop_day: 83 (days cycle)
EAU: 15 (ea/year)
Each order qty:4 (ea/order)
order frequency: 3.75 (times/year)
pricing bucket: 2_piece_bucket
unit cost: 47.68
material purchasing cost: 715.2 (dollars/year)
holding cost:13.92 (dollars/year)
logistic cost:28.5 (dollars/year)
total_landing_cost_optimal: 757.62 (dollars/year)

Material: 12640762-T01, HEAD,PIVOT BRG CARRIER
vop_day: 79 (days cycle)
EAU: 64 (ea/year)
Each order qty:16 (ea/order)
order frequency: 4.0 (times/year)
pricing bucket: 16_piece_bucket
unit cost: 58.29
material purchasing cost: 3730.56 (dollars/year)
h

In [85]:
result_path = filedialog.askdirectory()
result_file_name = os.path.split(file_path)[1]

In [86]:
result_path

'S:/OSK-Share/DEPT/LOGISTICS/LC3/Projects/Logistics Optimization Tool/Files for Nate to Upload - VOP/VOP_Result'

In [87]:
vop_master_run.to_excel(os.path.join(result_path, result_file_name), index=False)